# Introduction


This IPython notebook explains a basic workflow two tables using py_entitymatching. Our goal is to come up with a workflow to match books from amazon and goodread sites. Specifically, we want to create a matcher with precision of at least 90% and recall as high as possible. The datasets contain information about the books.

First, we need to import py_entitymatching package and other libraries as follows:

In [2]:
import sys
import py_entitymatching as em

Matching two tables typically consists of the following three steps:

1. Reading the input tables

2. Blocking the input tables to get a candidate set

3. Matching the tuple pairs in the candidate set

# Read input tables

In [4]:
# Get the paths
path_A = "../DATA/TableA.csv"
path_B = "../DATA/TableB.csv"
A = em.read_csv_metadata(path_A, key='ID')
B = em.read_csv_metadata(path_B, key='ID')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [5]:
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Number of tuples in A: 3000
Number of tuples in B: 3000
Number of tuples in A X B (i.e the cartesian product): 9000000


In [6]:
A.head(2)

,ID,title,author,rating,format
0,a1,To Kill a Mockingbird,Harper Lee,4.26,Paperback
1,a2,Harry Potter and the Sorcerer's Stone (Harry Potter #1),J.K. Rowling,4.45,Hardcover


In [7]:
B.head(2)

,ID,title,author,rating,format
0,b1,Make Your Bed: Little Things That Can Change Your Life...And Maybe the World,William H. McRaven,4.7,Hardcover
1,b2,The Silent Wife: A gripping emotional page turner with a twist that will take your breath away,Kerry Fisher,4.1,Paperback


In [8]:
# Display the keys of the input tables
em.get_key(A), em.get_key(B)

('ID', 'ID')

# Block tables to get candidate set

Before we do the matching, we would like to remove the obviously non-matching tuple pairs from the input tables. This would reduce the number of tuple pairs considered for matching.
For the matching problem at hand, we know that two books with different titles will not match. So we decide the apply blocking over title:

In [9]:
# Create overlap blocker
ob = em.OverlapBlocker()

# Block using title attribute
C1= ob.block_tables(A, B, 'title', 'title', word_level=True, overlap_size=3, l_output_attrs=['title','author'], r_output_attrs=['title','author'], show_progress=False)

In [10]:
len(C1)

54723

The number of tuple pairs are still huge. So, we would like to further reduce it. Knowing the dataset, we can say that two books with different authors will not match. Hence, we can block the candidate set of tuple pairs on author.

In [11]:
C2 = ob.block_candset(C1, 'author', 'author', word_level=True, overlap_size=1, show_progress=False)

In [12]:
len(C2)

548

# Debug blocker output

The number of tuple pairs considered for matching is reduced to 54723 (from 9000000), but we would want to make sure that the blocker did not drop any potential matches. We could debug the blocker output in py_entitymatching as follows:

In [13]:
# Debug blocker output
dbg = em.debug_blocker(C2, A, B, output_size=200)

In [14]:
# Display first few tuple pairs from the debug_blocker's output
dbg.head()

,_id,ltable_ID,rtable_ID,ltable_title,ltable_author,ltable_rating,ltable_format,rtable_title,rtable_author,rtable_rating,rtable_format
0,0,a33,b2990,Holy Bible: King James Version,Anonymous,4.43,Hardcover,"Holy Bible: King James Version, 1611 Edition",Hendrickson Publishers,4.4,Hardcover
1,1,a1861,b166,The Liars' Club,Mary Karr,3.93,Paperback,The Book Club,Mary Alice Monroe,4.3,Paperback
2,2,a2142,b2554,A Short History of Progress,Ronald Wright,4.11,Paperback,A Short History of Nearly Everything,Bill Bryson,4.6,Paperback
3,3,a120,b2197,The Lorax,Dr. Seuss,4.35,Hardcover,The Sneetches and Other Stories,Dr. Seuss,4.8,Hardcover
4,4,a1688,b2024,Fantastic Beasts and Where to Find Them,Newt Scamander,3.97,Hardcover,Fantastic Beasts and Where to Find Them: The Original Screenplay,J.K. Rowling,4.6,Hardcover


From the debug blocker's output we observe that the current blocker drops few potential matches. We would want to update the blocking sequence to avoid dropping these potential matches.
The potential matches are the ones where the book title length is less than 3 words. Hence, we could use block box blocker to create a rule to avoid blocking these tuple pairs. After comparing the length of the book titles, we used jacard measure on the attribute 'author'.

In [15]:
# Getting q-gram and jacard from the library
qgm_3 = em.get_tokenizers_for_blocking()['qgm_3']
jaccard = em.get_sim_funs_for_blocking()['jaccard']

In [16]:
# Defining the black box blocker rule
def bbRule(ltuple, rtuple):
    l_title = ltuple['title'].split()
    r_title = rtuple['title'].split()
    if len(l_title) < 3 and (len(r_title) == len(l_title)):
        for i in range(len(l_title)):
            if (l_title[i] != r_title[i]):
                return True
            if (jaccard(qgm_3(ltuple['author']), qgm_3(rtuple['author'])) < 0.5):
                return True
        return False
    else:
        return True

bb = em.BlackBoxBlocker()
# Setting the rule in black box blocker
bb.set_black_box_function(bbRule)

In [17]:
D = bb.block_tables(A, B, l_output_attrs=['title','author'], r_output_attrs=['title','author'])

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:05:49


In [18]:
len(D)

17

In [19]:
D.head(2)

,_id,ltable_ID,rtable_ID,ltable_title,ltable_author,rtable_title,rtable_author
0,0,a10,b2979,Jane Eyre,Charlotte Brontë,Jane Eyre,Charlotte Bronte
1,1,a28,b2987,Wuthering Heights,Emily Brontë,Wuthering Heights,Emily Bronte


In [20]:
# Combine blocker outputs
C = em.combine_blocker_outputs_via_union([C2, D])

In [21]:
len(C)

565

We observe that the number of tuple pairs considered for matching is increased to 565 (from 548). Now let us debug the blocker output again to check if the current blocker sequence is dropping any potential matches.

In [22]:
# Debug again
dbg = em.debug_blocker(C, A, B)

In [23]:
# Display first few rows from the debugger output
dbg.head(3)

,_id,ltable_ID,rtable_ID,ltable_title,ltable_author,ltable_rating,ltable_format,rtable_title,rtable_author,rtable_rating,rtable_format
0,0,a33,b2990,Holy Bible: King James Version,Anonymous,4.43,Hardcover,"Holy Bible: King James Version, 1611 Edition",Hendrickson Publishers,4.4,Hardcover
1,1,a120,b2197,The Lorax,Dr. Seuss,4.35,Hardcover,The Sneetches and Other Stories,Dr. Seuss,4.8,Hardcover
2,2,a2142,b2554,A Short History of Progress,Ronald Wright,4.11,Paperback,A Short History of Nearly Everything,Bill Bryson,4.6,Paperback


We observe that the current blocker sequence does not drop obvious potential matches, and we can proceed with the matching step now. A subtle point to note here is, debugging blocker output practically provides a stopping criteria for modifying the blocker sequence.

# Matching tuple pairs in the candidate 

In this step, we would want to match the tuple pairs in the candidate set. Specifically, we use learning-based method for matching purposes. This typically involves the following five steps:

    Sampling and labeling the candidate set
    Splitting the labeled data into development and evaluation set
    Selecting the best learning based matcher using the development set
    Evaluating the selected matcher using the evaluation set

As the number of tuple pairs after blocking are only 565, we can skip sampling the candidate set. Also we have manually labeled the tuple pairs which have survived blocking.

In [7]:
#Loading manually labeled data
G = em.read_csv_metadata("../DATA/Labelled Set G.csv", key="_id", fk_ltable="ltable_ID", fk_rtable="rtable_ID", ltable=A, rtable=B)
len(G)

Metadata file is not present in the given path; proceeding to read the csv file.


565

## Splitting the labeled data into development and evaluation set

In this step, we split the labeled data into two sets: development (I) and evaluation (J). Specifically, the development set is used to come up with the best learning-based matcher and the evaluation set used to evaluate the selected matcher on unseen data.

In [54]:
# Split S into development set (I) and evaluation set (J)
# Label I and J
IJ = em.split_train_test(G, train_proportion=0.66, random_state=29)
I = IJ['train']
J = IJ['test']

## Selecting the best learning-based matcher

Selecting the best learning-based matcher typically involves the following steps:

    Creating a set of learning-based matchers
    Creating features
    Converting the development set into feature vectors
    Selecting the best learning-based matcher using k-fold cross validation
    
### Creating a set of learning-based matchers

In [26]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0, probability=True)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='Naive Bayes')

### Creating features

Next, we need to create a set of features for the development set. py_entitymatching provides a way to automatically generate features based on the attributes in the input tables. 

In [27]:
atypes1 = em.get_attr_types(A)
atypes2 = em.get_attr_types(B)

In [28]:
block_c = em.get_attr_corres(A, B)
# Only title and author are sufficient to learn the matcher
block_c['corres'] = [('title', 'title'), ('author', 'author')]

In [29]:
# Creating tokenizers and similarity functions
tok = em.get_tokenizers_for_blocking()
sim = em.get_sim_funs_for_blocking()

In [30]:
# Generate features
feature_table = em.get_features(A, B, atypes1, atypes2, block_c, tok, sim)

In [31]:
# List the names of the features generated
feature_table['feature_name']

0           title_title_jac_qgm_3_qgm_3
1       title_title_cos_dlm_dc0_dlm_dc0
2                       title_title_mel
3                  title_title_lev_dist
4                   title_title_lev_sim
5         author_author_jac_qgm_3_qgm_3
6     author_author_cos_dlm_dc0_dlm_dc0
7     author_author_jac_dlm_dc0_dlm_dc0
8                     author_author_mel
9                author_author_lev_dist
10                author_author_lev_sim
11                    author_author_nmw
12                     author_author_sw
Name: feature_name, dtype: object

### Converting the development set to feature vectors

In [55]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, feature_table=feature_table, attrs_after='label',show_progress=False)
# Impute feature vectors with the mean of the column values.
H = em.impute_table(H, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], strategy='mean')

In [56]:
# Display first few rows
H.head(3)

,_id,ltable_ID,rtable_ID,title_title_jac_qgm_3_qgm_3,title_title_cos_dlm_dc0_dlm_dc0,title_title_mel,title_title_lev_dist,title_title_lev_sim,author_author_jac_qgm_3_qgm_3,author_author_cos_dlm_dc0_dlm_dc0,author_author_jac_dlm_dc0_dlm_dc0,author_author_mel,author_author_lev_dist,author_author_lev_sim,author_author_nmw,author_author_sw,label
185,185,a1961,b689,1.000000,1.000000,1.000000,0.0,1.000000,0.608696,0.816497,0.666667,0.947368,5.0,0.736842,9.0,14.0,1
176,176,a189,b1012,0.078125,0.235702,0.735376,82.0,0.211538,0.225806,0.408248,0.250000,0.831818,13.0,0.409091,-1.0,9.0,0
121,121,a15,b1477,0.250000,0.490290,0.840823,44.0,0.488372,0.571429,0.816497,0.666667,0.941176,5.0,0.705882,7.0,12.0,0


### Selecting the best matcher using cross-validation

Now, we select the best matcher using k-fold cross-validation. For the purposes of this guide, we use five fold cross validation and use 'precision' and 'recall' metric to select the best matcher.

In [57]:
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], k=5, target_attr='label', metric_to_select_matcher='f1', random_state=0)
result['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.885933,0.880029,0.881771
1,RF,0.901088,0.903844,0.901685
2,SVM,0.915229,0.839137,0.875228
3,LinReg,0.883766,0.929070,0.904290
4,LogReg,0.856335,0.918478,0.884201
5,Naive Bayes,0.802324,0.877938,0.837078


We observe that the requied precision of at least 90% and high recall is achieved by Random Forest. Hence, debugging a matcher is not required.

## Evaluating the matching output

Evaluating the matching outputs for the evaluation set typically involves the following four steps:

    Converting the evaluation set to feature vectors
    Training matcher using the feature vectors extracted from the development set
    Predicting the evaluation set using the trained matcher
    Evaluating the predicted matches

### Converting the evaluation set to feature vectors

As before, we convert to the feature vectors (using the feature table and the evaluation set)

In [58]:
# Convert the J into a set of feature vectors using F
L = em.extract_feature_vecs(J, feature_table=feature_table, attrs_after='label',show_progress=False)
L = em.impute_table(H, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], strategy='mean')

In [59]:
# Training the matchers
dt.fit(table=H, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], target_attr='label')
svm.fit(table=H, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], target_attr='label')
lg.fit(table=H, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], target_attr='label')
ln.fit(table=H, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], target_attr='label')
rf.fit(table=H, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], target_attr='label')
nb.fit(table=H, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], target_attr='label')

### Predicting the matches

Next, we predict the matches for the evaluation set (using the feature vectors extracted from it).

In [60]:
predictionsDT = dt.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], append=True, target_attr='predicted', inplace=False, return_probs=True, probs_attr='proba')
predictionsRF = rf.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], append=True, target_attr='predicted', inplace=False, return_probs=True, probs_attr='proba')
predictionsSVM = svm.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], append=True, target_attr='predicted', inplace=False, return_probs=True, probs_attr='proba')
predictionsLinReg = ln.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], append=True, target_attr='predicted', inplace=False, return_probs=True, probs_attr='proba')
predictionsLogReg = lg.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], append=True, target_attr='predicted', inplace=False, return_probs=True, probs_attr='proba')
predictionsNB = nb.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], append=True, target_attr='predicted', inplace=False, return_probs=True, probs_attr='proba')

There is no proba function defined for Linear Regression Matcher in scikit learn. So we return the probs as 1


### Evaluating the predictions

Finally, we evaluate the accuracy of predicted outputs

In [62]:
eval_resultRF = em.eval_matches(predictionsRF, 'label', 'predicted')
em.print_eval_summary(eval_resultRF)

Precision : 98.92% (183/185)
Recall : 100.0% (183/183)
F1 : 99.46%
False positives : 2 (out of 185 positive predictions)
False negatives : 0 (out of 187 negative predictions)
